# SMAI Assignment - 2

## Question 1: Naive Bayes and Clustering

### Part 1: Naive Bayes

[Files](https://drive.google.com/drive/folders/1OUVrOMp2jSSBDJSqvEyXDFTrhiyZnqit?usp=sharing)

You will be performing Sentiment Analysis on a product review dataset with reviews from customers and star rating belonging to four classes (1,2,4,5). You can use sklearn for this question. Your tasks are as follows:

1.   Clean the text by removing punctations and preprocess them using techniques such as stop word removal, stemming etc. You can explore anything!
1.  Create BoW features using the word counts. You can choose the words that form the features such that the performance is optimised. Use the train-test split provided in `train_test_index.pickle` and report any interesting observations based on metrics such as accurarcy, precision, recall and f1 score (You can use Classification report in sklearn).
1. Repeat Task 2 with TfIdf features.

ModuleNotFoundError: No module named 'resolution'

In [1]:
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
with open('train_test_index.pickle', 'rb') as handle:
    train_test_index_dict = pickle.load(handle)

In [25]:
train_test_index_dict

{'train_index': [6914,
  8332,
  2791,
  25106,
  11135,
  24667,
  17806,
  25963,
  9915,
  8994,
  13636,
  5549,
  23229,
  10371,
  16618,
  18480,
  7943,
  21225,
  11262,
  11648,
  8155,
  9368,
  19936,
  8083,
  6122,
  2994,
  7960,
  14536,
  21716,
  2200,
  23441,
  3157,
  728,
  14015,
  12039,
  3925,
  21321,
  12676,
  1330,
  1517,
  12478,
  20823,
  15546,
  8719,
  19362,
  23878,
  581,
  22649,
  4046,
  25419,
  14805,
  14254,
  18461,
  21107,
  7717,
  20676,
  17548,
  22053,
  13985,
  17399,
  13462,
  21006,
  11720,
  14037,
  15591,
  16001,
  16167,
  8725,
  22280,
  16528,
  14056,
  14723,
  11239,
  13893,
  22314,
  8562,
  19913,
  25693,
  11292,
  9216,
  25676,
  10837,
  11490,
  5713,
  10523,
  14606,
  5458,
  23303,
  19400,
  11665,
  21757,
  17438,
  1779,
  12292,
  25538,
  20682,
  26374,
  26634,
  2809,
  18530,
  19996,
  9708,
  2091,
  6652,
  3082,
  16330,
  26136,
  17355,
  21364,
  14707,
  18816,
  19887,
  15551,
  19

In [3]:
print(train_test_index_dict.keys())


dict_keys(['train_index', 'test_index'])


In [4]:
train_indices = train_test_index_dict['train_index']
test_indices = train_test_index_dict['test_index']

In [5]:
import pandas as pd

data = pd.read_csv('product_reviews.csv')
data.head()

,text,stars,sentiment
0,Total bill for this horrible service? Over $8G...,1.0,0
1,Went in for a lunch. Steak sandwich was delici...,5.0,1
2,This place has gone down hill. Clearly they h...,1.0,0
3,"Walked in around 4 on a Friday afternoon, we s...",1.0,0
4,Michael from Red Carpet VIP is amazing ! I rea...,4.0,1


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhinavanand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
stemmer = PorterStemmer()

In [8]:
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
    return ' '.join(words)

In [9]:
data['cleaned_text'] = data['text'].apply(clean_text)

In [26]:
data

,text,stars,sentiment,cleaned_text
0,Total bill for this horrible service? Over $8G...,1.0,0,total bill horribl servic gs crook actual nerv...
1,Went in for a lunch. Steak sandwich was delici...,5.0,1,went lunch steak sandwich delici caesar salad ...
2,This place has gone down hill. Clearly they h...,1.0,0,place gone hill clearli cut back staff food qu...
3,"Walked in around 4 on a Friday afternoon, we s...",1.0,0,walk around friday afternoon sat tabl bar walk...
4,Michael from Red Carpet VIP is amazing ! I rea...,4.0,1,michael red carpet vip amaz reach need help pl...
...,...,...,...,...
26656,"Save your money, and don't go here.\nThis plac...",1.0,0,save money go place stuck went walk entir stor...
26657,My husband and I thought we would give The Sus...,2.0,0,husband thought would give sushi anoth tri sto...
26658,"Petit café sympa, peu de place assise. Bonne b...",4.0,1,petit caf sympa peu de place assis bonn bouff ...
26659,Absolutely delicious! The food was full of ama...,5.0,1,absolut delici food full amaz flavor owner qui...


# task 1 bag of word

In [10]:
from sklearn.feature_extraction.text import CountVectorizer


In [11]:
vectorizer = CountVectorizer(max_features=1500) 
X_bow = vectorizer.fit_transform(data['cleaned_text']).toarray()

In [27]:
X_bow.shape

(26661, 1500)

In [13]:
# split the data

X_train_bow, X_test_bow = X_bow[train_indices], X_bow[test_indices]
y_train, y_test = data['stars'][train_indices], data['stars'][test_indices]

In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [15]:
classifier = MultinomialNB()  # initialize the classifier
classifier.fit(X_train_bow,y_train)


MultinomialNB()

In [16]:
y_pred_bow = classifier.predict(X_test_bow)

In [17]:
print(classification_report(y_test, y_pred_bow))

              precision    recall  f1-score   support

         1.0       0.69      0.72      0.70      1149
         2.0       0.44      0.43      0.43       587
         4.0       0.47      0.60      0.53      1981
         5.0       0.83      0.74      0.78      5082

    accuracy                           0.68      8799
   macro avg       0.61      0.62      0.61      8799
weighted avg       0.70      0.68      0.69      8799



# task 2 using tfidf 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tfidf_vectorizer = TfidfVectorizer(max_features=1500)

In [20]:
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text']).toarray()

In [21]:
X_train_tfidf, X_test_tfidf = X_tfidf[train_indices], X_tfidf[test_indices]

In [22]:
# Train with TF-IDF
classifier.fit(X_train_tfidf, y_train)   # i use the same MultinomialNB classifier as before.

MultinomialNB()

In [23]:
y_pred_tfidf = classifier.predict(X_test_tfidf)

In [24]:
print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

         1.0       0.80      0.59      0.68      1149
         2.0       0.62      0.05      0.10       587
         4.0       0.52      0.20      0.29      1981
         5.0       0.68      0.96      0.80      5082

    accuracy                           0.68      8799
   macro avg       0.65      0.45      0.47      8799
weighted avg       0.66      0.68      0.62      8799



### Part 2: Clustering

You will be performing kmeans clustering on the same product reviews dataset from Part 1. In this question, instead of statistically computing features, you will use the embeddings obtained from a neural sentiment analysis model (huggingface: siebert/sentiment-roberta-large-english).

You can use sklearn for this question. Your tasks are as follows:


1. Perform kmeans clustering using sklearn. Try various values for number of clusters (k) and plot the elbow curve. For each value of k, plot WCSS (Within-Cluster Sum of Square). WCSS is the sum of the squared distance between each point and the centroid in a cluster.
1. Perform task 1 with cluster initialisation methods [k-means++, forgy ("random" in sklearn)].
1. In this case, since the ground truth labels (star rating) are available we can evaluate the clustering using metrics like purity, nmi and rand score. Implement these metrics from scratch and evaluate the clustering. [Reference](https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-clustering-1.html)

In [ ]:
import gzip
import numpy as np

f = gzip.GzipFile('roberta_embeds.npy.gz', "r")
embeds = np.load(f)
print(embeds.shape)